# 최대 유량 (Maximum Flow)

`-` 간선에 용량이 존재하는 방향 그래프에서 소스에서 싱크까지 보낼 수 있는 최대 유량을 계산하자

## 도시 왕복하기 1

- 문제 출처: [백준 17412번](https://www.acmicpc.net/problem/17412)

`-` 네트워크 플로우를 사용하는 문제이다

`-` 생소한 내용이라 관련 자료를 많이 찾아봤다

`-` 참고: https://en.wikipedia.org/wiki/Flow_network

`-` 그래프 내에 증가 경로가 없을 때까지 유량을 흘려보내서 최대 유량을 계산할 수 있다

`-` 소스에서 싱크까지 흘려보낼 수 있는 최대 유량을 $f$라 하고 노드의 개수를 $V$, 간선의 개수를 $E$라고 하자

`-` 매 증가 경로마다 적어도 $1$의 유량을 흘리므로 증가 경로를 DFS로 찾는다면 시간 복잡도는 $O((V + E)f)$가 된다

`-` 그런데 보통 $V\le E$이므로 단순하게 $O(Ef)$라고 표현한다

`-` 참고: https://en.wikipedia.org/wiki/Ford-Fulkerson_algorithm

`-` 이렇게 해서 찾은 최대 유량이 진짜 최대 유량임을 증명하는 건 다음을 참고하자

`-` 참고: https://en.wikipedia.org/wiki/Max-flow_min-cut_theorem#Proof

`-` 한편, 증가 경로를 BFS로 찾는 방식을 Edmonds–Karp 알고리즘이라고 한다

`-` Edmonds–Karp 알고리즘의 시간 복잡도는 $\min\left(O(Ef), O\left(VE^2\right)\right)$이다

`-` 참고: https://en.wikipedia.org/wiki/Edmonds-Karp_algorithm

`-` 찾아본 자료를 보면 보통 코드가 길던데 개념적으로 접근하면 그리 복잡하진 않다

`-` 증가 경로를 찾기 위해 BFS를 사용하는데 BFS는 기본 그래프 알고리즘이다

`-` 증가 경로가 존재하는지 판단하는 건 소스에서 출발해 싱크까지 조건하에 BFS로 도달 가능한지 따지는 것이다

`-` 여기서 조건이란 간선의 유량이 간선의 용량을 초과하지 않는 것이다

`-` 증가 경로를 찾았으면 해당 경로를 따라 유량을 흘려보낸다

`-` 증가 경로 추적을 위해 역추적 배열을 만들고 BFS 안에서 갱신해야 된다

`-` 증가 경로 중 일부에 실제로는 존재하지 않는 역방향 간선이 있을 수 있다

`-` 역방향 간선은 흘러온 유량을 다시 되돌려 보낸다고 생각하면 된다

`-` 어떤 간선 $u\to v$를 통해 유량을 보냈다면 반대로 $v \to u$에는 그만큼의 유량을 차감해줘야 한다 (유량의 대칭성)

`-` 아무튼 증가 경로가 존재하지 않을 때까지 반복하면 최대 유량을 계산할 수 있다

`-` 이제 본 문제를 풀어보자

`-` $N$개의 도시가 $P$개의 단방향 길로 연결되어 있으니 방향 그래프로 구현해주면 된다

`-` $1$번에서 $2$번으로 가는 서로 다른 경로를 최대한 많이 찾아야 하는데 한 경로에 포함된 길이 다른 경로에 포함되면 안된다

`-` 이는 간선의 용량이 $1$인 것과 마찬가지다

`-` 따라서 모든 간선의 용량을 $1$로 설정한 후 소스를 $1$번 도시, 싱크를 $2$번 도시로 취급해 Edmonds–Karp 알고리즘을 사용하여 최대 유량을 계산하면 된다

`-` 최대 유량을 찾는 과정만 제대로 알고 있다면 구현은 비교적 간단하다

`-` 간선의 용량이 $1$이라 증가 경로에 있는 모든 간선들의 잔여 용량의 최솟값인 $\operatorname{bottleneck}$은 항상 $1$이지만 연습 차원에서 일반화된 코드로 작성했다

In [8]:
from collections import defaultdict, deque


def edmonds_karp(graph, source, sink, flow, capacity):
    max_flow = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        prevs = bfs(graph, source, sink, flow, capacity)
        exists_augmenting_path = prevs[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(prevs, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        max_flow += bottleneck
    return max_flow


def bfs(graph, source, sink, flow, capacity):
    queue = deque([source])
    visited = {source: True}
    prevs = [NONE] * (N + 1)
    while queue:
        u = queue.popleft()
        for v in graph[u]:
            residual_capacity = capacity[u][v] - flow[u][v]
            if v in visited or residual_capacity <= 0:
                continue
            prevs[v] = u
            if v == sink:
                return prevs
            visited[v] = True
            queue.append(v)
    return prevs


def track_augmenting_path(prevs, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = prevs[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1] 
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1] 
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global N, NONE, INF
    N, P = map(int, input().split())
    INF = float("inf")
    NONE = -1
    source = 1
    sink = 2
    flow = [[0] * (N + 1) for _ in range(N + 1)]
    capacity = [[1] * (N + 1) for _ in range(N + 1)]
    graph = defaultdict(list)
    for _ in range(P):
        start, end = map(int, input().split())
        graph[start].append(end)  # 정방향
        graph[end].append(start)  # 역방향
        capacity[end][start] = 0  # 역방향 간선의 용량은 0
    max_flow = edmonds_karp(graph, source, sink, flow, capacity)
    print(max_flow)


solution()

# input
# 5 5
# 1 3
# 3 2
# 1 5
# 5 4
# 4 2

 5 5
 1 3
 3 2
 1 5
 5 4
 4 2


2


## 도시 왕복하기 2

- 문제 출처: [백준 2316번](https://www.acmicpc.net/problem/2316)

`-` 간선뿐만 아니라 정점도 중복될 수 없는 조건 하에 최대 유량을 계산하는 문제라고 한다

`-` [도시 왕복하기 1](https://www.acmicpc.net/problem/17412) 문제와 달리 한 번 방문한 도시는 다시 방문할 수 없다

`-` 또한 간선이 양방향이고 $1$번 도시와 $2$번 도시를 왔다 갔다 할 수 있는 최대 횟수를 고려해야 한다

`-` 일단 간선이 양방향이므로 $1$번 도시에서 $2$번 도시로 이동하는 것만 고려해도 된다

`-` $1$번 도시에서 $2$번 도시로 가는 경로를 뒤집으면 $2$번 도시에서 $1$번 도시로 가는 것이므로 왔다 갔다 하는 것이 된다

`-` [도시 왕복하기 1](https://www.acmicpc.net/problem/17412) 문제에선 경로를 한 번만 사용하기 위해 간선의 용량을 $1$로 설정했었다

`-` 하지만 노드에는 용량을 설정할 수 없다

`-` 한 번 방문한 노드에 마킹을 해두고 마킹된 노드는 방문하지 않도록 해볼 수 있다

`-` 하지만 그렇게 하면 그래프 형식이 달라져 Edmonds-Karp 알고리즘으로 최대 유량을 찾을 수 있을지가 의문이다

`-` $3\to 5, 4 \to 5, 5 \to 6, 5 \to 7$인 그래프를 고려해보자 (원래는 양방향인데 증가 경로는 단방향이니까 단방향 그래프라 생각하자)

`-` $3\to 5$ 간선이든 $4\to 5$ 간선이든 하나를 선택하면 다른 간선은 사용할 수 없다

`-` Edmonds-Karp 알고리즘을 사용할 수 있도록 일반적인 그래프를 유지하면서 한 번 사용된 노드는 다시 사용되지 않도록 해야 한다

`-` 여기서 기막힌 묘수가 떠올랐는데 더미 노드와 더미 간선을 만들어주면 된다

`-` $5$번 노드의 더미 노드 $105$번 노드와 더미 간선 $5 \to 105$과 $105 \to 6, 105\to 7$을 만들자

`-` 이 간선들은 있으나 없으나 최대 유량에 영향을 주지 않는다

`-` 예컨대 $3\to 5 \to 105 \to 6$은 사실 $3 \to 5 \to 6$과 동일하다

`-` 그런데 $5 \to 105$와 $105 \to 6, 105\to 7$이 생김으로써 $5$번 노드가 한 번 선택되면 $5 \to 105$ 간선은 사용이 불가능해진다

`-` 그러면 $5$번 도시에서 $6,7$번 도시에 갈 수 없게 되며 이는 도시를 한 번만 방문하게 만든 것과 동일하다

`-` 그렇게 하면서도 그래프 형식을 훼손하지 않았다 (일반적인 그래프와 동일함)

`-` $N$개의 도시가 존재하고 도시의 번호는 $1$부터 $N$까지이다

`-` $i + N$번 더미 도시를 만든 뒤 $i$번 노드와 $i + N$번 노드를 잇는 단방향 간선을 추가하자

`-` 단, 소스인 $1$번 도시와 싱크인 $2$번 도시는 여러 번 방문해도 되므로 소스와 싱크에 대해서는 더미 노드를 만들지 않아야 한다

`-` $i$번 도시와 $j$번 도시를 잇는 길이 존재한다면 $i + N$번 도시와 $j$번 도시를 단방향으로 이으면 된다

`-` 길이 양방향이므로 $j$번 도시와 $i+N$번 도시도 단방향으로 연결해주자

`-` 그래프가 정의됐으니 Edmonds-Karp 알고리즘으로 소스에서 싱크까지 흘려보낼 수 있는 최대 유량을 계산하면 된다

`-` $90\%$에서 틀려서 반례 하나 봤다

`-` 소스($1$)와 싱크($2$)가 아닌 $x$번 노드를 고려하자

`-` $x \to 2$ 간선이 입력으로 주어지면 나는 단순히 $x$번 노드에 $2$를 추가해서 그래프를 구성했다

`-` 이렇게 하면 안되고 $x + N$번 노드에 $2$를 추가해줘야 한다 

`-` 그래야 $x \to x + N$ 간선이 활성화되고 $x$번 도시는 한 번만 사용되게 된다

`-` 이 외에 삽질을 더 했지만 $96\%$에서 틀려서 반례 하나 더 봤다

`-` 나는 $i$번 도시와 $j$번 도시가 입력으로 들어오면 $i + N$번 도시에 $j$번 도시를 추가하고 $j + N$번 도시에 $i$번 도시를 추가했다 (단방향)

`-` 길이 양방향이라 단순하게 $i + N \to j$ 간선의 역방향 간선이 $j + N \to i$ 간선이라 생각했다

`-` 이렇게만 하면 $i + N$번 도시는 $j$번 도시로 갈 수 있지만 $j$번 도시에선 $i + N$번 도시로 갈 수 없어 유량을 되돌려보낼 수 없다 ($j+N$번 도시와 $i$번 도시도 마찬가지)

`-` 따라서 역방향 간선을 정의해주고 해당 간선의 용량을 $0$으로 만들어줘야 한다

`-` 원래의 그래프는 양방향 간선이라 역방향 간선을 정의하는게 애매했지만 나는 그래프에 더미 노드와 더미 간선을 추가해 변경시켰다

`-` 소스와 싱크를 제외한 임의의 노드는 항상 바로 다음에 더미 노드를 지나야 하고 이는 단방향 간선이다

`-` 즉, $j \to i + N$ 간선과 $i \to j + N$ 간선을 역방향 간선으로 추가하고 용량을 $0$으로 설정하면 된다

`-` $i + N \to i$ 간선도 용량이 $0$인 역방향 간선으로 추가해야 한다

`-` 이렇게 하면 소스와 싱크를 제외한 노드에선 다음 동선이 항상 더미 노드가 되어 도시를 한 번만 방문하게 된다

`-` 또한 간선의 용량이 $0$인 역방향 간선을 만들어 줌으로써 흘러온 유량을 다시 되돌려 보낼 수 있도록 했다

`-` 간선이 양방향이라 복잡한 문제였다

In [143]:
from collections import defaultdict, deque


def edmonds_karp(graph, source, sink, flow, capacity):
    max_flow = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        prevs = bfs(graph, source, sink, flow, capacity)
        exists_augmenting_path = prevs[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(prevs, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        max_flow += bottleneck
    return max_flow


def bfs(graph, source, sink, flow, capacity):
    queue = deque([source])
    visited = {source: True}
    prevs = [NONE] * (2 * N + 1)
    while queue:
        u = queue.popleft()
        for v in graph[u]:
            residual_capacity = capacity[u][v] - flow[u][v]
            if v in visited or residual_capacity <= 0:
                continue
            prevs[v] = u
            if v == sink:
                return prevs
            queue.append(v)
            visited[v] = True
    return prevs


def track_augmenting_path(prevs, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = prevs[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def add_dummy_edges(graph, capacity):
    for i in range(3, N + 1):
        graph[i].append(i + N)  # 정방향
        graph[i + N].append(i)  # 역방향
        capacity[i + N][i] = 0  # 역방향 간선의 용량은 0


def solution():
    global N, NONE, INF
    N, P = map(int, input().split())
    NONE = -1
    INF = float("inf")
    source = 1
    sink = 2
    flow = [[0] * (2 * N + 1) for _ in range(2 * N + 1)]
    capacity = [[1] * (2 * N + 1) for _ in range(2 * N + 1)]
    graph = defaultdict(list)
    add_dummy_edges(graph, capacity)
    for _ in range(P):
        u, v = map(int, input().split())  # u -> v, v -> u
        if v < u:
            u, v = v, u
        if u == 1:
            graph[u].append(v)
            continue
        if u == 2:
            graph[v + N].append(u)
            continue
        for i, j in [[u, v], [v, u]]:
            graph[i + N].append(j)  # 정방향
            graph[j].append(i + N)  # 역방향
            capacity[j][i + N] = 0  # 역방향 간선의 용량은 0
    max_flow = edmonds_karp(graph, source, sink, flow, capacity)
    print(max_flow)


solution()

# input
# 5 6
# 1 5
# 1 4
# 4 5
# 3 4
# 4 2
# 3 2

 5 6
 1 5
 1 4
 4 5
 3 4
 4 2
 3 2


1


`-` 이러한 방식을 정점 분할이라 하는 것 같더라

`-` 들어오는 정점과 나가는 정점으로 분할한 뒤 두 정점 사이에 용량이 $1$인 간선을 추가하면 도시 방문 횟수를 제한할 수 있다

`-` 만약 $2$번까지 방문하는 걸 허용한다면 용량이 $2$인 간선을 사이에 추가하면 될 것이다

## 파티

- 문제 출처: [백준 2367번](https://www.acmicpc.net/problem/2367)

`-` 네트워크 플로우 알고리즘을 연습해보자

`-` 사람과 음식을 그래프상에서 노드로 나타내자

`-` 각 사람이 요리할 수 있는 음식을 단반향 간선으로 연결하면 된다

`-` 이때 한 사람당 같은 종류의 음식은 한 접시만 가져갈 수 있으므로 간선의 용량은 $1$이어야 한다

`-` 소스와 사람을 간선으로 연결할 건데 한 명당 최대 $K$ 접시를 가져갈 수 있으므로 간선의 용량은 $K$로 설정하면 된다

`-` 또, 음식과 싱크를 간선으로 연결할 건데 각 음식의 종류마다 가져올 수 있는 양의 제한으로 간선의 용량을 설정하면 된다

`-` 사람 번호와 음식 번호를 다르게 해야 노드가 겹치지 않는다

`-` 사람 노드에는 $1$번부터 $N$번까지 번호를 매기고 음식 노드에는 $N + 1$번부터 $N + D$번까지 번호를 매기겠다

`-` 소스는 $0$번, 싱크는 $N+D+1$번 노드로 설정하자

`-` 그래프가 준비됐으니 Edmonds-Karp 알고리즘을 통해 최대 유량을 찾으면 된다

`-` 참고로 소스와 싱크에 대해선 용량이 $0$인 역방향 간선을 정의하지 않아도 된다

`-` 소스는 출발점이므로 이미 방문한 상태라 다시 방문할 수 없어 역방향 간선이 필요 없다

`-` 싱크를 방문하면 bfs가 종료되므로 싱크에서 역방향 간선을 통해 다른 노드로 이동할 수 없다

In [2]:
from collections import defaultdict, deque


def edmons_karp(graph, source, sink, flow, capacity, num_nodes):
    max_flow = 0
    exists_augmenting_path = True
    while exists_augmenting_path:
        prevs = bfs(graph, source, sink, flow, capacity, num_nodes)
        exists_augmenting_path = prevs[sink] != NONE
        if not exists_augmenting_path:
            break
        augmenting_path = track_augmenting_path(prevs, sink)
        bottleneck = compute_bottleneck(augmenting_path, flow, capacity)
        update_flow(augmenting_path, flow, bottleneck)
        max_flow += bottleneck
    return max_flow
        

def bfs(graph, source, sink, flow, capacity, num_nodes):
    queue = deque([source])
    visited = [False] * num_nodes
    visited[source] = True
    prevs = [NONE] * num_nodes
    while queue:
        u = queue.popleft()
        for v in graph[u]:
            residual_capacity = capacity[u][v] - flow[u][v]
            if visited[v] or residual_capacity <= 0:
                continue
            prevs[v] = u
            if v == sink:
                return prevs
            visited[v] = True
            queue.append(v)
    return prevs


def track_augmenting_path(prevs, sink):
    augmenting_path = []
    node = sink
    while node != NONE:
        augmenting_path.append(node)
        node = prevs[node]
    augmenting_path.reverse()
    return augmenting_path


def compute_bottleneck(augmenting_path, flow, capacity):
    bottleneck = INF
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        bottleneck = min(capacity[u][v] - flow[u][v], bottleneck)
    return bottleneck


def update_flow(augmenting_path, flow, bottleneck):
    for i in range(len(augmenting_path) - 1):
        u, v = augmenting_path[i], augmenting_path[i + 1]
        flow[u][v] += bottleneck
        flow[v][u] -= bottleneck


def solution():
    global INF, NONE
    N, K, D = map(int, input().split())
    food_limits = [0] + list(map(int, input().split()))
    INF = float("inf")
    NONE = -1
    source = 0
    sink = N + D + 1
    num_nodes = N + D + 2
    flow = [[0] * num_nodes for _ in range(num_nodes)]
    capacity = [[1] * (num_nodes) for _ in range(num_nodes)]
    graph = defaultdict(list)
    graph[source] = list(range(1, N + 1))
    capacity[source] = [K] * num_nodes
    for food in range(N + 1, N + D + 1):
        graph[food].append(sink)
        capacity[food][sink] = food_limits[food - N]
    for person in range(1, N + 1):
        food_list = list(map(lambda x: int(x) + N, input().split()))[1:]
        graph[person] = food_list
        for food in food_list:
            graph[food].append(person)
            capacity[food][person] = 0
    max_flow = edmons_karp(graph, source, sink, flow, capacity, num_nodes)
    print(max_flow)


solution()

# input
# 4 3 5
# 2 2 2 2 3
# 4 1 2 3 4
# 4 2 3 4 5
# 3 1 2 4
# 3 1 2 3

 4 3 5
 2 2 2 2 3
 4 1 2 3 4
 4 2 3 4 5
 3 1 2 4
 3 1 2 3


9
